# Neural Network - Similiarity Function

This notebook details the process of choosing the similarity function that determines the images that are similar to a given image.
The implementation of the network is based on the Siamese network implementation. This class of networks is known to be more robust to class imbalance, so it fits the data on which we train.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F



In [ ]:
# load the data
data = pd.read_csv('data.csv')

# split the data into training and testing
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),
        )
        
        # Adaptive pooling layer to ensure the output size is consistent
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6))  # Adjust pooling size to handle dynamic input
        
        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(256 * 6 * 6, 1024),  # Input size adjusted for adaptive pooling
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, 2)
        )
        
    def forward_once(self, x):
        # Forward pass 
        x = self.cnn1(x)
        x = self.adaptive_pool(x)  # Adaptive pooling to standardize the feature map size
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = self.fc1(x)
        return x

    def forward(self, input1, input2):
        # Forward pass of input 1
        output1 = self.forward_once(input1)
        # Forward pass of input 2
        output2 = self.forward_once(input2)
        return output1, output2



In [ ]:
# # define the model
# class SiameseNetwork(nn.Module):
#     def __init__(self):
#         super(SiameseNetwork, self).__init__()
#         # Setting up the Sequential of CNN Layers
#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(1, 96, kernel_size=11,stride=1),
#             nn.ReLU(inplace=True),
#             nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
#             nn.MaxPool2d(3, stride=2),
            
#             nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
#             nn.ReLU(inplace=True),
#             nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
#             nn.MaxPool2d(3, stride=2),
#             nn.Dropout2d(p=0.3),

#             nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
#             nn.ReLU(inplace=True),
            
#             nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(3, stride=2),
#             nn.Dropout2d(p=0.3),
#         )
#         # Defining the fully connected layers
#         self.fc1 = nn.Sequential(
#             nn.Linear(30976, 1024),
#             nn.ReLU(inplace=True),
#             nn.Dropout2d(p=0.5),
            
#             nn.Linear(1024, 128),
#             nn.ReLU(inplace=True),
            
#             nn.Linear(128,2))
        
#     def forward_once(self, x):
#         # Forward pass 
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         # forward pass of input 1
#         output1 = self.forward_once(input1)
#         # forward pass of input 2
#         output2 = self.forward_once(input2)
#         return output1, output2

In [ ]:
# define the loss function
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on:
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x0, x1, y):
        # euclidian distance
        diff = x0 - x1
        dist_sq = torch.sum(torch.pow(diff, 2), 1)
        dist = torch.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = torch.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
        loss = torch.sum(loss) / 2.0 / x0.size()[0]
        return loss

In [ ]:
# training loop
network = SiameseNetwork().cuda()
# Decalre Loss Function
criterion = ContrastiveLoss()
# Declare Optimizer
optimizer = optim.Adam(network.parameters(), lr=1e-3, weight_decay=0.0005)
def train_siamese_network(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (img1, img2, labels) in enumerate(train_loader):
            # Move tensors to the appropriate device
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if (i + 1) % 10 == 0:  # Print every 10 batches
                print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}")
                running_loss = 0.0

    return model

model = train_siamese_network()
torch.save(model.state_dict(), "model.pt")

In [ ]:
# # training loop
# network = SiameseNetwork().cuda()
# # Decalre Loss Function
# criterion = ContrastiveLoss()
# # Declare Optimizer
# optimizer = optim.Adam(network.parameters(), lr=1e-3, weight_decay=0.0005)
# #train the model
# def train():
#     loss=[] 
#     counter=[]
#     iteration_number = 0
#     for epoch in range(1,config.epochs):
#         for i, data in enumerate(train_dataloader,0):
#             img0, img1 , label = data
#             img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
#             optimizer.zero_grad()
#             output1,output2 = network(img0,img1)
#             loss_contrastive = criterion(output1,output2,label)
#             loss_contrastive.backward()
#             optimizer.step()    
#         print("Epoch {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
#         iteration_number += 10
#         counter.append(iteration_number)
#         loss.append(loss_contrastive.item())
#     plt.show_plot(counter, loss)   
#     return network
# #set the device to cuda
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = train()
# torch.save(model.state_dict(), "model.pt")
# print("Model Saved Successfully") 

In [ ]:
siamese_net = SiameseNetwork().to(device)
criterion = ContrastiveLoss(margin=1.0)
optimizer = optim.Adam(siamese_net.parameters(), lr=0.001)

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

# Train the model
num_epochs = 100
train_siamese_network(siamese_net, train_loader, criterion, optimizer, num_epochs)